[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juansensio/blog/blob/master/082_sensio_copilot/082_sensio_copilot.ipynb)

# Sensio CoPilot

En este post vamos a entrenar una red neuronal para generación de código similar al funcionamiento de [github Copilot](https://copilot.github.com/). Llevo usando Copilot unos meses y la verdad que puedo decir que es una herramienta increíble, que ha aumentado considerablemente mi productividad como programador. Hace unos días leí [este](https://twitter.com/lvwerra/status/1467933794699259908?s=21) hilo en Twitter y creí que sería interesante intentar replicar un sistema como Copilot desde cero. Así que sin más dilación, ¡vamos a ello!

Github Copilot utiliza el modelo conocido como [Codex](https://arxiv.org/abs/2107.03374), desarrollado por OpenAI. Este modelo está basado en la arquitectura GPT (de lo que hablaremos más adelante) y *tuneado* con código público extraído de Github. Esto significa que el modelo fue pre-entrenado de manera no supervisada con mucho texto y luego se hizo *fine tuning* con el código extraído de Github para la tarea de generación de texto (nuevo código autocompletado). En contraste, nosotros entrenaremos un modelo desde cero para la tarea de generación de texto con un dataset preparado para ello a modo de demostración.

## El Dataset

El dataset usado por OpenAI para entrenar Codex fue extraído de 54 millones de reposiotrios públicos de Github, conteniendo 179 GB de archivos Python de menos de 1 MB. Tras varias etapas de procesado, el dataset final ocupó 159 GB. Como no tenemos acceso a este dataset, usaremos [CodeParrot](https://huggingface.co/datasets/lvwerra/codeparrot-clean), un dataset elaborado por [HuggingFace](https://huggingface.co/) para la tarea de generación de código.

> Recuerda que OpenAI al final tiene que ganar dinero de alguna manera, y ésta es cobrando por el uso de sus modelos a través de la API. Siendo el modelo público, su única ventaja competitiva reside en los datos usados durante el entrenamiento. Esto es una tendencia clara en el mundo del Software 2.0, dónde el valor real está en los datos y no en el código (aunque como diría Andrej Karpathy en el Software 2.0 los datos SON el código y el modelo no es más que el binario resultante de la compilación del mismo, lo que llamamos el proceso de entrenamiento).

El dataset ocupa unos 50 GB, aproximadamente una tercera parte del dataset usado originalmente por OpenAI. ¡Nada mal! Puedes descargarlo utilizando los siguientes comandos:

```
git clone https://huggingface.co/datasets/lvwerra/codeparrot-clean-train
git clone https://huggingface.co/datasets/lvwerra/codeparrot-clean-valid
```

> Para poder descargarlos necesitarás instalar [Git LFS](https://git-lfs.github.com/).

In [23]:
from pathlib import Path
from glob import glob

path = Path('data/codeparrot-clean-train')

files = glob(str(path) + '/*.json.gz')
len(files), files[:3]

(52,
 ['codeparrot-clean-train/file-000000000007.json.gz',
  'codeparrot-clean-train/file-000000000053.json.gz',
  'codeparrot-clean-train/file-000000000026.json.gz'])

In [19]:
import pandas as pd 

sample = pd.read_json(files[2], lines=True)
sample

,repo_name,path,copies,size,content,license,hash,line_mean,line_max,alpha_frac,autogenerated
0,jalavik/inspire-next,setup.py,1,4558,# -*- coding: utf-8 -*-\n#\n# This file is par...,gpl-2.0,-4849180608980663294,27.848101,77,0.615840,False
1,dlzhangxg/cloud-ml-sdk,cloud_ml_samples/keras/mnist/trainer/task.py,1,2967,"# Copyright 2017 Xiaomi, Inc.\n#\n# Licensed u...",apache-2.0,-1822461891537938192,30.231579,74,0.649815,False
2,openstack/heat,heat/engine/support.py,1,2683,"#\n# Licensed under the Apache License, Ver...",apache-2.0,-1815098437948811103,36.788732,78,0.622438,False
3,imapp-pl/golem,tests/golem/network/test_golem_protocol.py,1,1444,import unittest\nfrom devp2p.service import Wi...,gpl-3.0,-5671972220290336808,37.000000,77,0.654432,False
4,willimoa/pydal,pydal/dialects/mongo.py,1,22083,"import re\nfrom .._compat import PY2, basestri...",bsd-3-clause,7148857323817256703,34.389423,93,0.518951,False
...,...,...,...,...,...,...,...,...,...,...,...
99995,georgestarcher/TA-SyncKVStore,bin/ta_synckvstore/cloudconnectlib/core/ext.py,1,10300,import calendar\nimport json\nimport re\nimpor...,mit,-2116068727318744484,29.654762,80,0.593495,False
99996,nabucosound/django-propaganda,propaganda/migrations/0001_initial.py,1,2828,# -*- coding: utf-8 -*-\nfrom __future__ impor...,bsd-3-clause,2285667758777388556,38.830986,114,0.541372,False
99997,znuxor/aoc2016,4.py,1,41871,#!/usr/bin/python3\nimport operator\n\n# room ...,bsd-3-clause,-2104829268388077325,40.662687,118,0.834659,False
99998,sharadagarwal/autorest,AutoRest/Generators/Python/Python.Tests/Expect...,1,5365,# coding=utf-8\n# ----------------------------...,mit,-2646046330546511516,35.250000,110,0.630009,False


In [24]:
sample.content[10]

'#!/usr/bin/env python\n\n# Copyright (C) 2014 Aldebaran Robotics\n#\n# Licensed under the Apache License, Version 2.0 (the "License");\n# you may not use this file except in compliance with the License.\n# You may obtain a copy of the License at\n#\n#     http://www.apache.org/licenses/LICENSE-2.0\n#\n# Unless required by applicable law or agreed to in writing, software\n# distributed under the License is distributed on an "AS IS" BASIS,\n# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n# See the License for the specific language governing permissions and\n# limitations under the License.\n#\n#import ROS dependencies\nimport rospy\n\n#import NAO dependencies\nfrom naoqi_driver.naoqi_node import NaoqiNode\nfrom geometry_msgs.msg import PoseStamped\nfrom geometry_msgs.msg import Pose\nimport almath\nimport tf\nfrom tf.transformations import euler_from_quaternion\n\nclass MoveToListener(NaoqiNode):\n\n    def __init__(self):\n        NaoqiNode.__init__(self, \'

# El Modelo

El modelo usado por OpenAI, Codex, está basado en la arquitectura [GPT](https://paperswithcode.com/method/gpt) y contiene 12 billones de parámetros. Esto está un poco fuera de nuestra alcance (de momento 😝) así que usaremos la implementación de Karpathy, [minGPT](https://github.com/karpathy/minGPT), que nos permite entrenar pequeños transformers basados en la arquitectura GPT.

## Entrenamiento

Fit one sample con función def sum(a, b): return a+b

Fit one batch del dataset

We train Codex using the same learning rate as the corresponding GPT model, with a 175 step linear warmup and
cosine learning rate decay. We train for a total of 100 billion
tokens, using the Adam optimizer with β1 = 0.9, β2 = 0.95, eps = 10−8, and a weight decay coefficient of 0.1

## Validación

Generar texto

## Resumen

Mejoras: usar tokenizer pre-entrenado (gpt3), usar modelo más grande, ...
Next: meter en una api (por ejemplo heroku), hacer vscode extension, ...